In [ ]:
import torch
from torch_geometric.data import Data
import networkx as nx
from torch_geometric.utils import to_undirected
from torch_geometric.datasets import GNNBenchmarkDataset

In [ ]:
"""Monoidal operation $\circ$"""
def mon_op(A,B):
    return A+B+torch.mm(A,B)

In [ ]:
"""Monoidal operation $\circ$"""
def mon_op(A,B):
    return A+B+(torch.sparse.mm(A.to_sparse(),B)).to_dense()

In [ ]:
"""Computing Image for all nodes"""
def image(X,n,m):
    cover=torch.zeros(n,n,n)
    for i in range(n):
        
        dec=torch.clone(X)
        cover[i].t()[i]=X.t()[i]
        dec[i]=0
        dec.t()[i]=0
        M=torch.zeros(n,n)
        N=torch.ones(n,n)
        for k in range(n):
            if cover[i][k].sum()!=0:
                M.t()[k]=1
                N.t()[k]=0
        c=0
            #M.sum()!=0  c<m 
        while M.sum()!=0:
            cover[i]=mon_op((M*dec)-(((M*dec)*((M*dec).t()))),cover[i])
            dec=dec-(M*dec)
            M=torch.zeros(n,n)
            N=torch.ones(n,n)
            for k_ in range(n):
                if cover[i][k_].sum()!=0 and dec.t()[k_].sum()!=0:
                    M.t()[k_]=1
                    N.t()[k_]=0
            c+=1
    return cover

In [ ]:
dataset = GNNBenchmarkDataset(root='/tmp/CSL',name='CSL')

In [ ]:
l_dataset=len(dataset)

In [ ]:
Distinguished_graphs=[]
for graph in range(l_dataset):
    gr=dataset[graph]
    num_nodes=gr.num_nodes
    num_edges=gr.num_edges
    Ad_mat=torch.zeros(num_nodes,num_nodes)
    node_int=torch.ones(10,num_nodes)
    for edge in range(num_edges):
        Ad_mat[gr.edge_index[0][edge]][gr.edge_index[1][edge]]=1
    
    Image=image(Ad_mat,num_nodes,5)
    
    
    s=torch.sum(torch.sum(Image))
    
    
    if s not in Distinguished_graphs:
        Distinguished_graphs.append(s)

In [ ]:
"""Number of graphs that model can not distinguish"""
num_of_different_isomorph_classes=10
print(num_of_different_isomorph_classes-len(Distinguished_graphs))
